In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 17.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import numpy as np
from ultralytics import YOLO 
from keras.models import load_model
import pandas as pd

In [3]:
yolo_model = YOLO('/kaggle/input/yolo-model/best.pt')
cnn_model = load_model('/kaggle/input/cnn-model/cnn_model.keras')

In [5]:
reverse_labels_dict = {0: 'N29', 1: 'D52', 2: 'N35', 3: 'U15', 4: 'M16', 5: 'G37', 6: 'T28', 7: 'Y2', 8: 'W18', 9: 'T20', 10: 'N37', 11: 'M23', 12: 'D54', 13: 'H6', 14: 'T30', 15: 'M18', 16: 'S24', 17: 'U33', 18: 'I9', 19: 'F40', 20: 'U1', 21: 'V13', 22: 'R4', 23: 'G35', 24: 'O50', 25: 'D60', 26: 'M195', 27: 'W24', 28: 'D10', 29: 'F18', 30: 'Aa26', 31: 'O31', 32: 'I10', 33: 'O28', 34: 'G40', 35: 'D28', 36: 'E1', 37: 'G39', 38: 'O49', 39: 'Q7', 40: 'F34', 41: 'G7', 42: 'V28', 43: 'F9', 44: 'D2', 45: 'L1', 46: 'E34', 47: 'M12', 48: 'S29', 49: 'F16', 50: 'G4', 51: 'N36', 52: 'N14', 53: 'Q3', 54: 'M29', 55: 'O34', 56: 'G26', 57: 'D36', 58: 'Y3', 59: 'F4', 60: 'G29', 61: 'P1', 62: 'S28', 63: 'N30', 64: 'D35', 65: 'V31', 66: 'M42', 67: 'W25', 68: 'W19', 69: 'D4', 70: 'N31', 71: 'Q1', 72: 'D46', 73: 'O1', 74: 'M8', 75: 'U28', 76: 'G14', 77: 'P8', 78: 'N25', 79: 'Z11', 80: 'G36', 81: 'D1', 82: 'N1', 83: 'D19', 84: 'M44', 85: 'N18', 86: 'M41', 87: 'Z7', 88: 'M20', 89: 'M40', 90: 'P6', 91: 'G21', 92: 'D58', 93: 'Z1', 94: 'F35', 95: 'T22', 96: 'G17', 97: 'N17', 98: 'X1', 99: 'M1', 100: 'N5', 101: 'T21', 102: 'D39', 103: 'G43', 104: 'E9', 105: 'G1', 106: 'V30', 107: 'M17', 108: 'W11', 109: 'R8', 110: 'G5', 111: 'U7', 112: 'F13', 113: 'G25', 114: 'Aa27', 115: 'Aa15', 116: 'V24', 117: 'Y5', 118: 'N41', 119: 'D56', 120: 'F26', 121: 'S34', 122: 'D21', 123: 'E23', 124: 'V4', 125: 'F31', 126: 'X8', 127: 'M3', 128: 'O4', 129: 'V7'}

input_width, input_height = 128, 128

In [6]:
def preprocess_image_for_cnn(image):
    resized_image = cv2.resize(image, (input_width, input_height))
    resized_image = resized_image / 255.0
    return np.expand_dims(resized_image, axis=0)

In [7]:
def detect_and_classify(image_path):
    image = cv2.imread(image_path)

    results = yolo_model(image) 
    bboxes = results[0].boxes.xyxy.cpu().numpy()

    predicted_labels = []

    for bbox in bboxes:
        x1, y1, x2, y2 = map(int, bbox) 
        cropped_image = image[y1:y2, x1:x2]  

        preprocessed_image = preprocess_image_for_cnn(cropped_image)
        prediction = cnn_model.predict(preprocessed_image)
        predicted_class_idx = np.argmax(prediction, axis=1)[0]

        predicted_label = reverse_labels_dict[predicted_class_idx]
        predicted_labels.append(predicted_label)

    return predicted_labels

In [ ]:
image_path = '/kaggle/input/newimg95/img95.png'
labels = detect_and_classify(image_path)
print("Predicted Hieroglyph Labels:", labels)



0: 640x544 4 hieroglyphs, 7.8ms
Speed: 1.8ms preprocess, 7.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 544)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Hieroglyph Labels: 'G17', 'O28', 'O49', 'N5'


In [20]:
df = pd.read_csv('/kaggle/input/hieroglyph-dict/hieroglyphs.csv')
filtered_df = df[df['Gardiner Code'].isin(labels)]
descriptions = filtered_df['Description'].tolist()

print(descriptions)

sentence = f"Create a meaningful sentence using the following words: {', '.join(description)}."
print(sentence)

['owl', 'column', 'village', 'sun']
Create a meaningful sentence using the following words: owl, column, village, sun.


In [21]:
pip install transformers

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [23]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [24]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2LMHeadModel.from_pretrained('gpt2-large', pad_token_id=tokenizer.eos_token_id)

In [25]:
tokenizer.eos_token_id

50256


In [26]:
tokenizer.decode(tokenizer.eos_token_id)

<|endoftext|>


In [28]:
numeric_id = tokenizer.encode(sentence, return_tensors = 'pt')
numeric_id

tensor([[ 6042,   962,    71,    11,  5457, 10200,   257,  6450,   282,    12, 26418,  4930, 27105,   286,   281,   393, 28391]])


In [33]:
result = model.generate(numeric_id, max_length = 50, num_beams = 5, no_repeat_ngram_size = 2, early_stopping = True)
 
generated_text = tokenizer.decode(result[0], skip_special_tokens = True)
generated_text 

'Create a meaningful sentence using the following words: owl, column, village, sun.The owl sits on a column near the village, basking in the sun.'
